In [38]:
import os
from dotenv import load_dotenv
from src.data.rag import retrieve_relevant_context
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import ChatOllama
from langchain_core.tools import tool


llm = ChatOllama(
    model="gemma3",
)

In [39]:
@tool
def submit_ticket(name: str, email: str) -> None:
    """Submits a Jira Ticket once all required information is passed"""
    print("Hooray, a ticket has been created for {name} with email {email}")

llm.bind_tools([submit_ticket])

RunnableBinding(bound=ChatOllama(model='gemma3'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'submit_ticket', 'description': 'Submits a Jira Ticket once all required information is passed', 'parameters': {'properties': {'name': {'type': 'string'}, 'email': {'type': 'string'}}, 'required': ['name', 'email'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [40]:
system_prompt = """
You are a frontdesk assistant for Capital Area Food Bank (CAFB).

A partner is creating a new issue of type: Delivery Issues/Damaged Goods.
Help them by gathering more information about their issue so that a Jira Ticket can be created. You need to get their name and email. Once you have their name and email, you can create Jira Ticket using functions

You have access to functions. If you decide to invoke any of the function(s), you MUST put it in the format of
[func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]

You SHOULD NOT include any other text in the response if you call a function
[
  {
    "name": "submit_ticket",
    "description": "Submits a Jira Ticket once all required information is passed",
    "parameters": {
      "type": "object",
      "properties": {
        "name": {
          "type": "string"
        },
        "email": {
          "type": "string"
        }
      },
      "required": [
        "name",
        "email"
      ]
    }
  }
]


DO NOT MAKE UP OR SIMULATE INFORMATION.
""".replace("{", "{{").replace("}", "}}")

question = """
Hello, I need help with my damaged goods. My name is Harsh and my email is patil@gmail.com. Please help me
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

In [42]:
output = chain.invoke({"question": question,})
output

AIMessage(content='[submit_ticket(name="Harsh", email="patil@gmail.com")]', additional_kwargs={}, response_metadata={'model': 'gemma3', 'created_at': '2025-04-15T04:27:24.588369491Z', 'done': True, 'done_reason': 'stop', 'total_duration': 762345564, 'load_duration': 56928887, 'prompt_eval_count': 325, 'prompt_eval_duration': 38407237, 'eval_count': 20, 'eval_duration': 665215670, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-023e0d84-043d-40cb-985b-8d5ab3ed866c-0', usage_metadata={'input_tokens': 325, 'output_tokens': 20, 'total_tokens': 345})

In [44]:
import re
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

# 1. Define the tool
@tool
def submit_ticket(name: str, email: str) -> str:
    """Submits a Jira Ticket once all required information is passed"""
    print(f"Hooray, a ticket has been created for {name} with email {email}")
    return f"Ticket created for {name} with {email}"

# 2. Use ChatOllama directly (no .with_tools)
llm = ChatOllama(model="gemma3")

# 3. Prompt construction
system_prompt = """
You are a frontdesk assistant for Capital Area Food Bank (CAFB).

A partner is creating a new issue of type: Delivery Issues/Damaged Goods.
Help them by gathering more information about their issue so that a Jira Ticket can be created. You need to get their name and email. Once you have their name and email, you can create Jira Ticket using functions.

You have access to functions. If you decide to invoke any of the function(s), you MUST put it in the format of
[submit_ticket(name="...", email="...")]
You SHOULD NOT include any other text in the response if you call a function.
""".replace("{", "{{").replace("}", "}}")

question = "Hello, I need help with my damaged goods. My name is Harsh and my email is patil@gmail.com. Please help me"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])

# 4. Run the LLM
chain = prompt | llm
response = chain.invoke({"question": question})

# 5. Manually parse for function call
match = re.search(r'submit_ticket\(name="(.+?)", email="(.+?)"\)', response.content)
if match:
    name = match.group(1)
    email = match.group(2)
    result = submit_ticket.invoke({"name": name, "email": email})
    print("Tool Output:", result)
else:
    print("No tool call detected.")


Hooray, a ticket has been created for Harsh with email patil@gmail.com
Tool Output: Ticket created for Harsh with patil@gmail.com


In [46]:
import os
from dotenv import load_dotenv
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import ChatOllama
from langchain_core.tools import tool

# Set up the LLM
llm = ChatOllama(
    model="mistral-small",
)

@tool
def submit_ticket(name: str, email: str) -> str:
    """Submits a Jira Ticket once all required information is passed"""
    print(f"Hooray, a ticket has been created for {name} with email {email}")
    return f"Successfully created a ticket for {name} with email {email}"

# Bind tools to the LLM
llm_with_tools = llm.bind_tools([submit_ticket])

# Modified system prompt to use standard tool calling format
system_prompt = """
You are a frontdesk assistant for Capital Area Food Bank (CAFB).

A partner is creating a new issue of type: Delivery Issues/Damaged Goods. Help them by gathering more information about their issue so that a Jira Ticket can be created. You need to get their name and email.

Once you have their name and email, you MUST use the submit_ticket tool to create a Jira Ticket.
"""

question = """
Hello, I need help with my damaged goods. My name is Harsh and my email is patil@gmail.com. Please help me
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}"),
])

# Create the chain with the LLM that has tools bound to it
chain = prompt | llm_with_tools

# Run the chain
output = chain.invoke({"question": question})
print(output)

content='' additional_kwargs={} response_metadata={'model': 'mistral-small', 'created_at': '2025-04-15T04:31:15.298583717Z', 'done': True, 'done_reason': 'stop', 'total_duration': 22079905837, 'load_duration': 10600946065, 'prompt_eval_count': 188, 'prompt_eval_duration': 1528294533, 'eval_count': 28, 'eval_duration': 9945355378, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-e8d723e7-958c-489a-b163-269709e0f65b-0' tool_calls=[{'name': 'submit_ticket', 'args': {'email': 'patil@gmail.com', 'name': 'Harsh'}, 'id': '623098d6-d5ba-4334-94b9-47287d7ccd7e', 'type': 'tool_call'}] usage_metadata={'input_tokens': 188, 'output_tokens': 28, 'total_tokens': 216}


In [48]:
output.tool_calls

[{'name': 'submit_ticket',
  'args': {'email': 'patil@gmail.com', 'name': 'Harsh'},
  'id': '623098d6-d5ba-4334-94b9-47287d7ccd7e',
  'type': 'tool_call'}]